In [1]:
import pandas as pd
from utils.config import domainConfigClass
from typing import List
from collections import Counter
from datetime import datetime


def generateInvoiceId(date: datetime, locationCode: str, invoiceVersion: int):
    return f'{date.strftime("%d%m%Y")}U{locationCode}{invoiceVersion}'


def generatePONo(date: datetime, storeId: str, supplierId: str):
    return f'{date.strftime("%Y%m%d")}-{storeId}-{supplierId}'


def nameExtracter(rightNamesList: List[str], wrongName: str) -> str:
    chances = [0] * len(rightNamesList)

    counterOriginal = Counter(wrongName.lower())
    for i, rightName in enumerate(rightNamesList):
        counterLocation = Counter(rightName.lower())
        chances[i] = (
            sum((counterOriginal & counterLocation).values())
            * 2
            / (sum(counterOriginal.values()) + sum(counterLocation.values()))
        )
    return rightNamesList[chances.index(max(chances))]

In [2]:
df = pd.read_excel("file.xlsx", sheet_name="Sheet1")

In [3]:
dropedDf = df.dropna(how="all")[
    [
        column.rawSheet.columnName
        for column in domainConfigClass.columns
        if column.rawSheet is not None
    ]
]

In [4]:
dropedDf.columns = [
    column.invoicePdf.columnName
    for column in domainConfigClass.columns
    if column.rawSheet is not None
]

In [5]:
pdfColumns = list(
    map(
        lambda x: x.columnName,
        sorted(
            [
                column.invoicePdf
                for column in domainConfigClass.columns
                if column.invoicePdf is not None and column.invoicePdf.index is not None
            ],
            key=lambda x: x.index,
        ),
    )
)

extraColumns = [
    column.invoicePdf.columnName
    for column in domainConfigClass.columns
    if column.rawSheet is None and column.invoicePdf is not None
]

date = datetime.strptime(str(df["Date"][0]), "%Y-%m-%d 00:00:00")
invoiceVersion = 1

pdfDF = dropedDf.copy()

for column in extraColumns:
    pdfDF[column] = ""

# pdfDF = pdfDF[pdfColumns]

pdfDF["Dispatched Qty"] = pdfDF["Dispatched Qty"].astype(int)
pdfDF["Rate"] = pdfDF["Rate"].astype(int)

pdfDF["Total Amount"] = pdfDF["Dispatched Qty"] * pdfDF["Rate"]

pdfDF["Location"] = pdfDF["Location"].apply(
    lambda x: nameExtracter(
        [location.locationName for location in domainConfigClass.locations], x
    )
)

In [6]:
tableDf = {
    location.locationName: {
        "dataFrame": pdfDF[pdfDF["Location"] == location.locationName][
            pdfColumns
        ].reset_index(drop=True),
        "invoiceNumber": generateInvoiceId(date, location.code, invoiceVersion),
        "poNo": generatePONo(date, location.storeId, domainConfigClass.supplierId),
        "shippingAddress": location.shippingAddress,
        "retailer": location.retailer,
    }
    for location in domainConfigClass.locations
}

In [44]:
curLoc = "Byramji"
current = tableDf["Byramji"]
with pd.ExcelWriter("./output.xlsx", engine="xlsxwriter") as xlW:
    currentDf = current["dataFrame"]
    currentDf["Sr"] = range(1, len(currentDf) + 1)
    currentDf.to_excel(
        xlW, sheet_name=current["invoiceNumber"], startrow=5, index=False
    )

    workBook = xlW.book
    workSheet = xlW.sheets[current["invoiceNumber"]]

    headerCellFormat = workBook.add_format(
        {
            "bold": True,
            "align": "center",
            "valign": "vcenter",
            "text_wrap": True,
        }
    )

    workSheet.merge_range(0, 0, 0, 2, current["retailer"], headerCellFormat)
    workSheet.merge_range(1, 0, 1, 2, curLoc, headerCellFormat)
    workSheet.merge_range(
        2, 0, 4, 2, f'Shipping Address: {current["shippingAddress"]}', headerCellFormat
    )
    # workSheet.write(2, 0, )

    workSheet.merge_range(0, 3, 0, 7, "Vendor Name: Upgrade Mandi", headerCellFormat)
    workSheet.merge_range(
        1, 3, 1, 7, f'Date: {date.strftime("%d/%m/%Y")}', headerCellFormat
    )
    workSheet.merge_range(
        2, 3, 2, 7, f'Invoice: {current["invoiceNumber"]}', headerCellFormat
    )
    workSheet.merge_range(
        3, 3, 3, 7, "Email: ankushmisal7387@gmail.com", headerCellFormat
    )
    workSheet.merge_range(4, 3, 4, 7, f'PO No: {current["poNo"]}', headerCellFormat)

    # workSheet.write(4, 1, date.strftime("%d-%m-%Y"))
    # workSheet.write(5, 0, "Sr")
    workSheet.set_column("A:A", 5)
    workSheet.set_column("B:F", 20)
    workSheet.set_column("G:G", 10)
    workSheet.set_column("H:H", 20)
    # workSheet.write(5, 1, "Article Name")
    # workSheet.write(5, 2, "UoM")

    pass